In [1]:
import os
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
google_vec_file = '~/Downloads/GoogleNews-vectors-negative300.bin'
google_model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)
import pickle
import numpy as np
import pandas as pd

#### Load Data

In [6]:
with open('mergedmess.pkl','rb') as file:
    mergedmess = pickle.load(file)

mergedmess.head()

,datemess,messages,rp_page,su_follow,botActive,botCreated,datepage,fans,pagename,firstContactDate,lastContactDate,gender,timezone,locale
0,2018-07-26 05:28:12.281,Yes,1748596732049183,1773780432666739,False,True,2017-07-19 13:33:14.479,1349,JokeBot,2017-11-17T04:33:49.265Z,2017-11-17T04:33:49.373Z,male,2.0,en_US
1,2018-07-26 05:28:12.280,Yes,1748596732049183,1773780432666739,False,True,2017-07-19 13:33:14.479,1349,JokeBot,2017-11-17T04:33:49.265Z,2017-11-17T04:33:49.373Z,male,2.0,en_US
2,2018-07-26 10:00:03.962,no,1748596732049183,1714087635268143,False,True,2017-07-19 13:33:14.479,1349,JokeBot,2017-11-17T05:55:53.814Z,2017-11-17T07:19:19.688Z,female,8.0,en_US
3,2018-07-27 08:15:06.328,More Puns,1748596732049183,1714087635268143,False,True,2017-07-19 13:33:14.479,1349,JokeBot,2017-11-17T05:55:53.814Z,2017-11-17T07:19:19.688Z,female,8.0,en_US
4,2018-07-27 08:15:20.982,Give me more puns!,1748596732049183,1714087635268143,False,True,2017-07-19 13:33:14.479,1349,JokeBot,2017-11-17T05:55:53.814Z,2017-11-17T07:19:19.688Z,female,8.0,en_US


### Vectorization with Google Gensim's Pre-trained W2V Model

In [7]:
def get_doc_vec(words, model):
    good_words = []
    for word in words:
        try:
            if model.wv[word] is not None:
                good_words.append(word)
        except:
            continue
    if len(good_words) == 0:
        return None
    return model.wv[good_words].mean(axis=0)

In [14]:
def google_vec(text):
    # Make a copy of the text dataframe for the Google work
    textual = text.copy()

    textual['messages'] = textual['messages'].str.lower()
    textual.drop_duplicates(inplace=True)

    # Retrieve the document vectors based on google word vectors
    textual_google_vecs = textual['messages'].str.split().map(lambda x: get_doc_vec(x, google_model))

    # Add to dataframe
    textual['vecs'] = textual_google_vecs

    # Drop the bad docs
    textual = textual.dropna()

    # Create a Numpy array of the document vectors
    textual_np_vecs = np.zeros((len(textual), 300))
    for i, vec in enumerate(textual['vecs']):
        textual_np_vecs[i, :] = vec

    return pd.concat([textual.reset_index().messages, pd.DataFrame(textual_np_vecs)], axis=1)

textual_google_data = google_vec(mergedmess)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


#### Store vectorized data.

In [ ]:
with open('google_text.pkl','wb') as file:
    pickle.dump(textual_google_data,file)